In [65]:
from pymongo import MongoClient
import requests
import pandas as pd
import json
import numpy as np

sklearn: 1.10.0+cpu


In [1]:
#aqui vai selecionar os agrupamentos e invocar o \classifier\learn

In [13]:
fdi_db = MongoClient('localhost', 27017).fdi
unknowns = fdi_db.unknown
classifier_knowledge = fdi_db.classifier_knowledge

In [54]:
# verifica os clusters que tem somente o valor especificado.

y = 4
only_y_list = []
for c in np.arange(0, 300):
    only_y = True
    ukn = list(unknowns.find({'cluster': int(c)}))

    for _, u in enumerate(ukn):
        if u['y'] != y:
            only_y = False
            break

    if only_y:
        only_y_list.append(int(c))



In [55]:
data = list(unknowns.find({'cluster': {"$in":only_y_list}}))
memory = list(classifier_knowledge.find({}))

In [56]:
url = 'http://127.0.0.1:5001/classifier'
headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json'}

def send(endpoint, params_dict):
    return requests.get(url+endpoint, data=json.dumps(params_dict), headers=headers).json()

In [57]:
sensors = []
concepts = []

for x in memory:
    sensors.append(list(x['sensor'].values()))
    concepts.append(x['concept'])

for x in data[:2000]:
    sensors.append(list(x['sensors'].values()))
    concepts.append(x['y'])

In [58]:
data_retrain = np.concatenate((np.array(sensors), np.array(concepts).reshape(-1,1)), axis=1)

np.random.shuffle(data_retrain)

In [59]:
for n, x in enumerate(data_retrain):
    r = send('/learn', {'x': pd.DataFrame(x[:-1]).T.to_json(), 'y': x[-1]})

In [111]:
for x in data:
    sensor = list(x['sensors'].values())
    r = send('/predict', {'x': pd.DataFrame(sensor).T.to_json(), 'y': y})
    print(r['y_hat'])

9
3
3
3
3
3
3
3
3
3
3


KeyboardInterrupt: 